# Introduction to Automation with LangChain, Generative AI, and Python
**2.4: LLM Writes a Book**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

This section will demonstrate how to effectively utilize the LLM to write a book. Writing a book is a uniquely complex challenge that requires planning and execution. Unlike simpler tasks, drafting a full-length book is a journey that extends beyond a single session with an LLM due to its extensive length and depth. However, with a systematic approach, you can harness the capabilities of an LLM to create a comprehensive manuscript, empowering you to conquer this challenge.

This section presents a method to craft a book using an LLM, which involves breaking down the book creation process into manageable segments. Our journey begins with a foundational step: selecting a subject for the book. Once a subject is chosen, the process unfolds through a series of iterative interactions with the LLM, each building upon the last to gradually construct the complete book, igniting your passion for writing.

Initially, we prompt the LLM to generate a random title based on the given subject. This title sets the thematic tone for the book and guides the subsequent steps. Following the title, we request the LLM to create a random synopsis. This synopsis acts as a narrative backbone, offering a brief overview of the book’s content and direction.

With a synopsis, the next step is to generate a table of contents. This table outlines the main chapters and sections of the book, providing a structured framework that organizes the content logically and cohesively. It is from this framework that the book starts to take shape.
The next phase involves creating each chapter. We engage in a focused session with the LLM for every chapter outlined in the table of contents. During each session, we provide the LLM with the synopsis and the complete table of contents to ensure consistency and continuity throughout the book. This iterative process allows each chapter to be crafted with attention to detail, ensuring it aligns with the previously established narrative and thematic elements.

By methodically iterating through these steps, we harness the capabilities of an LLM to transform a simple idea into a detailed and well-structured book. This chapter will guide you through each stage, providing practical advice on collaborating with an LLM to achieve more complex tasks.

We begin by accessing a large language model with a temperature of 0.7. We use a higher temperature to encourage creativity.

In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_aws import ChatBedrock
from IPython.display import display_markdown

MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'

# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.7},
)

We create a simple utility function to query the LLM with a custom system prompt. The system prompt tells the LLM that it is assisting in creating a book.

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

def query_llm(prompt):
  messages = [
      SystemMessage(
          content="""
You are writing a book, return just what I ask you for. Do not add comments, 
it is very important that you only provide the final output without any 
additional comments or remarks. Do not repeat my command or a summary of my
command.
"""
      ),
      HumanMessage(
          content=prompt
      ),
  ]

  output = llm.invoke(messages)
  return output.content


## Generate Title, Synopsis and Table of Contents

For this book, we allow the user to specify the subject specified by the SUBJECT variable. We then request the LLM to generate a random title based on this subject. It is essential that the prompt request that the LLM only the; LLMs often like to prefix with text such as "Here is a random title."

In [4]:
SUBJECT = "international spy story"

title = query_llm(f"""
Give me a random title for a book on the subject '{SUBJECT}'.
Return only the title, no additional text.
""").strip(" '\"")
print(title)

Rogue Operative: Betrayal at the Khyber Pass


Now that we have a title, we can request a random synopsis of the book.

In [5]:
synopsis = query_llm(f"""
Give me just a synopsis for a book of the title '{SUBJECT}' for a book on the subject '{SUBJECT}'.
""").strip(" '\"")
print(synopsis)

In a world of shifting alliances and hidden agendas, a daring secret agent must navigate a treacherous web of deceit and intrigue that spans continents. Caught in the crosshairs of rival intelligence agencies and powerful criminal organizations, they race against time to unravel a conspiracy that threatens global stability. With each heart-pounding twist and turn, this international spy story takes readers on a thrilling journey through exotic locales and high-stakes espionage, where trust is a luxury and one wrong move could be fatal.


Next, we generate the table of contents. For this generation, we provide all previous information. We also request a particular format for the table of contents. You may notice that I ask for the chapter numbers, even though they are an increasing numeric count and could easily be derived. This process works better because the LLM wants to provide the chapter numbers, and attempts to suppress chapter numbers are often inconsistent. It is easier to allow the LLM to generate chapter numbers but control where it generates them so that I can consistently remove them later.

In [6]:
toc = query_llm(f"""
Give me a table of contents for a book of the title '{title}' for a book on
the subject '{SUBJECT}' the book synopsis is '{synopsis}'.
Return the table of contents as a list of chapter titles.
Separate the chapter number and chapter title with a pipe character '|'.
Do not give me any line that is not a chapter!
""").strip(" '\"")
print(toc)

1|Whispers in the Night
2|The Khyber Gambit
3|Betrayal in Marrakech
4|Shadows in Moscow
5|The Hong Kong Connection
6|Echoes of Deception
7|Endgame in Istanbul
8|Secrets Unveiled
9|The Reckoning
10|A New Dawn


We must now parse the table of contents and remove the pipes and chapter numbers.

In [7]:
# Split the string into lines
lines = toc.splitlines()
print(lines)

# Extract titles using list comprehension
toc2 = [line.split('|')[1].strip() for line in lines if line]

# Print the list of titles
print(toc2)

['1|Whispers in the Night', '2|The Khyber Gambit', '3|Betrayal in Marrakech', '4|Shadows in Moscow', '5|The Hong Kong Connection', '6|Echoes of Deception', '7|Endgame in Istanbul', '8|Secrets Unveiled', '9|The Reckoning', '10|A New Dawn']
['Whispers in the Night', 'The Khyber Gambit', 'Betrayal in Marrakech', 'Shadows in Moscow', 'The Hong Kong Connection', 'Echoes of Deception', 'Endgame in Istanbul', 'Secrets Unveiled', 'The Reckoning', 'A New Dawn']


## Generate the Chapters of the Book

Next, we create a function capable of producing the text that makes up a chapter. To ensure that the function has enough context to generate each chapter, we provide the synopsis, the table of contents, and the chapter number. To test this code, we request that it develop a single chapter.

In [10]:
def render_chapter(num, chapter_title, title, subject, synopsis, toc):
  txt = query_llm(f"""
  Write Chapter {num}, titled "{chapter_title}" for a book of the title '{title}' for a book on
  the subject '{subject}' the book synopsis is '{synopsis}' the table of contents is '{toc}'.
  Give me only the chapter text, no chapter heading, no chapter title, number, no additional text.
  Make sure the chapter is at least 3,000 to 4,000 words. End with a complete sentence, do not cut off.
  """).strip(" '\"")
  return txt

txt = render_chapter(1, toc2[0], title, SUBJECT, synopsis, toc)
print(txt)

The night was still and silent, save for the faint whispers carried by the desert wind. In a remote outpost nestled in the rugged mountains of the Khyber Pass, a lone figure moved with practiced stealth, their footsteps leaving no trace on the sun-baked earth.

Agent Zara Khalil, one of the most skilled operatives in the covert world, had been tasked with a mission of utmost importance. Intelligence reports had hinted at a nefarious plot brewing in the shadows, one that threatened to destabilize the delicate balance of power in the region.

As she made her way through the narrow alleys and winding streets, Zara's senses were heightened, alert to the slightest movement or sound that could betray an enemy presence. Her years of training had honed her instincts to a razor's edge, allowing her to navigate the treacherous terrain with the grace of a seasoned predator.

Reaching her destination, a nondescript building that served as a front for a local arms dealer, Zara slipped through the s

We can now generate the entire book in Markdown, which allows some formatting. We begin by rendering the title and synopsis, the table of contents, and each chapter.

In [11]:
book = ""

# Render the title and synopsis
book += f"# {title}\n"
book += f"{synopsis}\n"

# Render the toc
book += f"\n## Table of Contents\n\n"
num = 1
for chapter_title in toc2:
  book += f"{num}. {chapter_title}\n"
  num += 1

# Render the book
chapter = 1
for chapter_title in toc2:
  print(f"Rendering chapter {chapter}/{len(toc2)}: {chapter_title}")
  txt = render_chapter(chapter, chapter_title, title, SUBJECT, synopsis, toc)
  book += f"\n\n## Chapter {chapter}: {chapter_title}\n"
  book += f"{txt}\n"
  chapter += 1

Rendering chapter 1/10: Whispers in the Night
Rendering chapter 2/10: The Khyber Gambit
Rendering chapter 3/10: Betrayal in Marrakech
Rendering chapter 4/10: Shadows in Moscow
Rendering chapter 5/10: The Hong Kong Connection
Rendering chapter 6/10: Echoes of Deception
Rendering chapter 7/10: Endgame in Istanbul
Rendering chapter 8/10: Secrets Unveiled
Rendering chapter 9/10: The Reckoning
Rendering chapter 10/10: A New Dawn


## Generate a PDF of the Book

Now that we have generated the book, we have saved it as a PDF.

In [12]:
import markdown
import pdfkit

# Convert Markdown to HTML
html = markdown.markdown(book)

# Convert HTML to PDF
pdfkit.from_string(html, 'output.pdf')

True